<p style="border: 1px solid #e7692c; border-left: 15px solid #e7692c; padding: 10px; text-align:justify;">
    <strong style="color: #e7692c">Tip.</strong> <a style="color: #000000;" href="https://nbviewer.jupyter.org/github/PacktPublishing/Hands-On-Computer-Vision-with-Tensorflow/blob/master/ch3/ch3_nb2_build_and_train_first_cnn_with_tf2.ipynb" title="View with Jupyter Online">Click here to view this notebook on <code>nbviewer.jupyter.org</code></a>. 
    <br/>These notebooks are better read there, as Github default viewer ignores some of the formatting and interactive content.
    </p>

<table style="font-size: 1em; padding: 0; margin: 0;">
    <tr style="vertical-align: top; padding: 0; margin: 0;">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #363636; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #e7692c;">Hands-on</span> Computer Vision with TensorFlow 2</span><br/>by <em>Eliot Andres</em> & <em>Benjamin Planche</em> (Packt Pub.)</strong><br/><br/>
        <strong>> Chapter 3: Modern Neural Networks</strong><br/>
    </p>

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #e7692c;">Notebook 2:</small><br/>Building and Training our First CNN<br/>with TensorFlow 2 and Keras</h1>
<br/>
<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    This notebook presents how to implement the <strong><em>LeNet-5</em> architecture</strong> as presented in the book, and applies it to <strong>hand-written digit recognition</strong> (on the MNIST dataset).
<br/>
<p style="border-left: 15px solid #e7692c; padding: 0 10px; text-align:justify;">
    <strong style="color: #e7692c;">Tip.</strong> The notebooks shared on this git repository illustrate some of notions from the book "<em><strong>Hands-on Computer Vision with TensorFlow 2</strong></em>" written by Eliot Andres and Benjamin Planche and published by Packt. If you enjoyed the insights shared here, <strong>please consider acquiring the book!</strong>
<br/><br/>
The book provides further guidance for those eager to learn about computer vision and to harness the power of TensorFlow 2 and Keras to build performant recognition systems for object detection, segmentation, video processing, smartphone applications, and more.</p>
        </td>
        <td style="vertical-align: top; padding: 0; margin: 0; width: 250px;">
    <a href="https://www.packtpub.com" title="Buy on Packt!">
        <img src="../banner_images/book_cover.png" width=250>
    </a>
    <p style="background: #e7692c; color:#ffffff; padding: 10px; text-align:justify;"><strong>Leverage deep learning to create powerful image processing apps with TensorFlow 2 and Keras. <br/></strong>Get the book for more insights!</p>
    <ul style="height: 32px; white-space: nowrap; text-align: center; margin: 0px; padding: 0px; padding-top: 10px;">
    <li style="display: inline-block; height: 100%; vertical-align: middle; float: left; margin: 5px; padding: 0px;">
        <a href="https://www.packtpub.com" title="Get the book on Amazon!">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_amazon.png" width="75px">
        </a>
    </li>
    <li style="display: inline-block; height: 100%; vertical-align: middle; float: left; margin: 5px; padding: 0px;">
        <a href="https://www.packtpub.com" title="Get your Packt book!">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_packt.png" width="75px">
        </a>
    </li>
    <li style="display: inline-block; height: 100%; vertical-align: middle; float: left; margin: 5px; padding: 0px;">
        <a href="https://www.packtpub.com" title="Get the book on O'Reilly Safari!">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_oreilly.png" width="75px">
        </a>
    </li>
    </ul>
        </td>
        </tr>
        </table>

In [1]:
import tensorflow as tf

## Preparing the Data

As presented in [Chapter 2](../ch2), we use Tensorflow and Keras helpers to load the commonly-used [MNIST](http://yann.lecun.com/exdb/mnist) training and testing datasets[$^1$](#ref) (Yann LeCun and Corinna Cortes hold all copyrights for this dataset). We also normalize the images (setting the pixel values from `[0, 255]` to `[0, 1]` and reshape them properly (as Tensorflow stores them as column-vectors):

In [2]:
num_classes = 10
img_rows, img_cols, img_ch = 28, 28, 1
input_shape = (img_rows, img_cols, img_ch)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

## Building and Training LeNet-5

We have demonstrated how CNNs can be implemented different ways depending on the _level of parametrization_ versus _succinctness_ one needs. In this case, we will use the Keras API to showcase once again how straightforward it makes implementing and using neural networks.

### Instantiating Convolutional Layers

In the previous notebook, we presented how one can perform convolutions on images. However, in neural networks, we want the convolutions' filters to be ***trainable***, and we may want to add a ***bias*** to the results and apply an ***activation function***.

We thus need to wrap the convolution operation into a `Layer` object, similar to how the fully-connected layer we implemented in Chapter 1 was built around the matrix operations.

TensorFlow 2/Keras provides its own `tf.keras.Layer` class we can extend. We demonstrate below how a simple convolution layer can be dfined this way:

In [4]:
class SimpleConvolutionLayer(tf.keras.layers.Layer):

    def __init__(self, num_kernels=32, kernel_size=(3, 3), strides=(1, 1), use_bias=True):
        """
        Initialize the layer.
        :param num_kernels:     Number of kernels for the convolution
        :param kernel_size:     Kernel size (H x W)
        :param strides:         Vertical and horizontal stride as list
        :param use_bias:        Flag to add a bias after covolution / before activation
        """
        # First, we have to call the `Layer` super __init__(), as it initializes hidden mechanisms:
        super().__init__()  
        # Then we assign the parameters:
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.strides = strides
        self.use_bias = use_bias

    def build(self, input_shape):
        """
        Build the layer, initializing its parameters according to the input shape.
        This function will be internally called the first time the layer is used, though
        it can also be manually called.
        :param input_shape:     Input shape the layer will receive (e.g. B x H x W x C)
        """
        # We are provided with the input shape here, so we know the number of input channels:
        num_input_channels = input_shape[-1]  # assuming shape format BHWC

        # Now we know how the shape of the tensor representing the kernels should be:
        kernels_shape = (*self.kernel_size, num_input_channels, self.num_kernels)

        # For this example, we initialize the filters with values picked from a Glorot distribution:
        glorot_uni_initializer = tf.initializers.GlorotUniform()
        self.kernels = self.add_weight(name='kernels',
                                       shape=kernels_shape,
                                       initializer=glorot_uni_initializer,
                                       trainable=True)  # and we make the variable trainable.

        if self.use_bias:  # If bias should be added, we initialize its variable too:
            self.bias = self.add_weight(name='bias',
                                        shape=(self.num_kernels,),
                                        initializer='random_normal',  # e.g., using normal distribution.
                                        trainable=True)

    def call(self, inputs):
        """
        Call the layer and perform its operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        # We perform the convolution:
        z = tf.nn.conv2d(inputs, self.kernels, strides=[1, *self.strides, 1], padding='VALID')

        if self.use_bias:  # we add the bias if requested:
            z = z + self.bias
        # Finally, we apply the activation function (e.g. ReLU):
        return tf.nn.relu(z)

    def get_config(self):
        """
        Helper function to define the layer and its parameters.
        :return:        Dictionary containing the layer's configuration
        """
        return {'num_kernels': self.num_kernels,
                'kernel_size': self.kernel_size,
                'strides': self.strides,
                'use_bias': self.use_bias}

Most of TensorFlow mathematical operations (e.g. in `tf.maths` and `tf.nn`) already have their derivatives defined by the framework. 
Therefore, as long as a layer is composed of such operations, **we do not have to manually define its backward propagation**. TensorFlow will automatically cover this, which saves a lot of efforts!

The convolutional layer we just implemented is therefore fully operational, and ready to be used inside a CNN, as we will demonstrate right away.

***Note:*** As convolutional layers are the most basic component of CNNs, TensorFlow obviously provides its own `tf.keras.layers.Conv2D` class. The module `tf.keras.layers` contains a large variety of pre-implemented standard layers, which we recommend to use whenever possible (as they have more advacned interfaces and optimized operations). For the sake of demonstration, we will still stick to our own simpler `SimpleConvolutionLayer` for the rest of this notebook, while using other Keras pre-defined layers.

### Implementing LeNet-5

***LeNet-5***[$^2$](#ref) is a simple CNN composed of 7 layers (2 *conv*, 2 *max-pool*, 3 *FC* + 1 helper layer to flatten the feature maps before the *FC*). For more details, we invite our readers to go back to Chapter 3.

Below, we thus present how one can extend the `tf.keras.Model` class to define a custom architecture:

In [5]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [6]:
class LeNet5(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet5, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        # self.conv1 = Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu')
        # self.conv2 = Conv2D(16, kernel_size=(5, 5), activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

***Note:*** For those who already experimented with [PyTorch](https://pytorch.org), another machine learning framework, this functional object-oriented approach to building neural networks may seem familiar. It has been adopted by TensorFlow 2.

## Classiyfing MNIST with a CNN


Now we can instantiate and compile our model for digit classification. To train it for this task, we instantiate the optimizer (a simple _SGD_ one for this example) and define the loss (the _categorical cross-entropy_).

***Note:*** The Keras loss named `"sparse_categorical_crossentropy"` performs the same cross-entropy operation as the `"categorical_crossentropy"`, but the former directly takes the ground-truth labels as inputs, while the latter requires the ground-truth labels to be _one-hotted_ before. Using the `"sparse_..."` loss thus saves us from manually having to transform the labels... 

In [7]:
model = LeNet5(num_classes)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Having sub-classed `tf.keras.Model`, our model has all its functionalities. For instance, we can call `model.summary()` to print a summary of its architecture:

In [8]:
# We can call `model.summary()` only if the model was built before. 
# It is normally done automatically at the first use of the network,
# inferring the input shapes from the samples the network is given.
# For instance, the command below would build the network (then use it for prediction):
_ = model.predict(x_test[:10])

# But we can build the model manually otherwise, providing the batched
# input shape ourselves:
batched_input_shape = tf.TensorShape((None, *input_shape))
model.build(input_shape=batched_input_shape)

# Method to visualize the architecture of the network:
model.summary()

Model: "le_net5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_convolution_layer (Si multiple                  156       
_________________________________________________________________
simple_convolution_layer_1 ( multiple                  2416      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  30840     
_________________________________________________________________
dense_1 (Dense)              multiple                  10164     
_________________________________________________________________
dense_2 (Dense)              multiple                  850 

Before launching the training, we also instantiate some Keras callbacks, i.e., utility functions automatically called at specific points during the training (before/after batch training, before/after a full epoch, etc.), in order to monitor it:

In [9]:
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]

(The Tensorboard callback allows us to monitor the training from Tensorboard. For that, open a console and launch the programm with the command "`tensorboard --logdir=./logs`". You can then access Tensorboard from a browser, via the URL "[`localhost:6006`](localhost:6006)".)

We can now pass everything to our model to train it:


In [10]:
history = model.fit(x_train, y_train,
                    batch_size=32, epochs=35, validation_data=(x_test, y_test), 
                    verbose=2,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)

Train on 60000 samples, validate on 10000 samples
Epoch 1/35
60000/60000 - 7s - loss: 1.8530 - accuracy: 0.4930 - val_loss: 0.8307 - val_accuracy: 0.8128
Epoch 2/35
60000/60000 - 6s - loss: 0.5104 - accuracy: 0.8630 - val_loss: 0.3465 - val_accuracy: 0.9029
Epoch 3/35
60000/60000 - 6s - loss: 0.3234 - accuracy: 0.9046 - val_loss: 0.2606 - val_accuracy: 0.9254
Epoch 4/35
60000/60000 - 6s - loss: 0.2641 - accuracy: 0.9212 - val_loss: 0.2223 - val_accuracy: 0.9357
Epoch 5/35
60000/60000 - 6s - loss: 0.2285 - accuracy: 0.9308 - val_loss: 0.1937 - val_accuracy: 0.9445
Epoch 6/35
60000/60000 - 6s - loss: 0.2031 - accuracy: 0.9390 - val_loss: 0.1749 - val_accuracy: 0.9500
Epoch 7/35
60000/60000 - 6s - loss: 0.1843 - accuracy: 0.9449 - val_loss: 0.1583 - val_accuracy: 0.9509
Epoch 8/35
60000/60000 - 6s - loss: 0.1692 - accuracy: 0.9491 - val_loss: 0.1475 - val_accuracy: 0.9569
Epoch 9/35
60000/60000 - 6s - loss: 0.1575 - accuracy: 0.9528 - val_loss: 0.1404 - val_accuracy: 0.9580
Epoch 10/35
60

## Conclusion

Given a machine with recent GPU(s), this training is quite fast (~0.1ms/image in our case). The final accuracy we obtain on the validation dataset (**~98%!**) is also much better compared to our previous attempts with simpler networks. Indeed, the relative error has been approximately divided by 5 (from ~5% to ~2% error), which is a significant improvement.

<a id="ref"></a>
#### References

1. LeCun, Y., Cortes, C., Burges, C., 2010. MNIST handwritten digit database. AT&T Labs [Online]. Available: http://yann.lecun.com/exdb/mnist 2, 18.
2. LeCun, Yann. "*LeNet-5, convolutional neural networks.*" [http://yann.lecun.com/exdb/lenet](http://yann.lecun.com/exdb/lenet) (2015): 20.